# Taking the messier data and making it nice


In [1]:
#!/usr/bin/python
# by DBR 2/2017 #

import pandas as pd
import numpy as np

#list the cohorts as they show up in excel
Contr = ['Z09087','Z09106','Z09192','A11201','Z09204'] #control
Trans = ['Z09144','Z08214','Z09125','A11200','Z09196'] #transplant
Nctlr = ['A11214']                                     #natural controller
Stabl = ['A11213','A11221']                            #stable set point no cART
CCR5T = ['A11219','R10155','T10187','R10159','T10173',
          'Z12037','Z12216','Z11151','Z12351','Z12370',
          'Z13133','Z12417','Z12420']                   #ccr5delta mutation transplant

cohorts     = [Contr,Trans,Nctlr,Stabl,CCR5T]
cohort_names= ['Control','Transplant','Natural Controller','Stable ART',r'CCR5$\Delta$ Transplant']
list_names  = [Contr+Trans+Nctlr+Stabl+CCR5T]
list_names  = list_names[0]
num_monkeys = len(list_names)

ATI_list = ['Z09087','Z09106','Z09192','Z09204','Z09144','Z08214','A11200','Z09196'] #8 good rebounds


/Applications/miniconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#import data from csv
df = pd.DataFrame.from_csv('VL.csv')

#find the primary infection
idx_ART = df[df['weeks'] == 24].index.tolist()[0] #the index of the earliest ART start for any monkey
idx_PI  = df[df['weeks'] == 0].index.tolist()[0]  #the index of the primary infection

#concatenate the 3 cohorts of interest
prim_df=pd.concat([df[cohorts[0]][idx_PI:idx_ART], 
                   df[cohorts[1]][idx_PI:idx_ART], 
                   df[cohorts[3]][idx_PI:idx_ART], 
                   df[cohorts[4]][idx_PI:idx_ART]], axis=1)

prim_weeks = df['weeks'][idx_PI:idx_ART] #get the weeks of primary infection


In [24]:
pd.DataFrame.to_csv(prim_df,'prim_df.csv')
pd.DataFrame.to_csv(pd.DataFrame(prim_weeks),'prim_weeks.csv')

In [4]:
names_in_order = ['Z09087', 'Z09144', 'Z09106', 'Z09192', 'Z08214', 'Z09125', 'A11200',
 'A11201', 'Z09196', 'Z09204', 'A11219', 'A11221', 'A11213', 'A11214', 'R10155',
 'T10187', 'R10159', 'T10173', 'Z12037', 'Z12216', 'Z11151', 'Z12351', 'Z12370',
 'Z13133', 'Z12417', 'Z12420']

In [28]:
#find the analytic treatment interruption viral loads
offART = [77,77,81,81,81,0,84,81,81,81,90,0,0,0,0,89,0,0,0,0,0,0,0,0,0,0]
ATI_df=pd.DataFrame(columns=ATI_list) #dataframe for ATI viral loads
ATI_weeks_df=pd.DataFrame(columns=ATI_list) #dataframe for ATI times

ATI_weeks={}

for i in range(len(names_in_order)):
    
    if names_in_order[i] in ATI_list:
    
        idx_ATI = df[df['weeks'] == offART[i]].index.tolist()[0] #find the index of ATI

        idx_Reb = idx_ATI #assume rebound at ATI
        
        #then adjust so reb at first positive 
        #print names_in_order[i]
        rebound=False
        #adjust index to be first positive index
        while rebound==False:
            if np.isnan(df[names_in_order[i]][idx_Reb]) or df[names_in_order[i]][idx_Reb]<=30:
                rebound=False
                idx_Reb+=1
            else:
                rebound=True
                idx_Reb-=1 #now start at index just before the first positive
            #print df[names_in_order[i]][idx_Reb] #print to check it is working
        
        #now fill up ATI dataframes
        ATI_df[names_in_order[i]]=df[names_in_order[i]][idx_Reb:]
        ATI_weeks_df[names_in_order[i]]=df['weeks'][idx_Reb:]
        ATI_weeks[names_in_order[i]]=df['weeks'][idx_Reb]#offART[i]


In [26]:
pd.DataFrame.to_csv(ATI_weeks_df,'ATI_weeks.csv')
pd.DataFrame.to_csv(ATI_df,'ATI_df.csv')